<a href="https://colab.research.google.com/github/CACMO/demo/blob/main/Calibracion_Tesec_Agent3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Analyze the semiconductor calibration results from '/content/manufacturing_data.CSV' by loading the data, performing initial data inspection (displaying first few rows, column names, and data types), calculating and displaying descriptive statistics for numerical columns, visualizing value distributions using histograms or box plots with appropriate legends, and finally, summarizing the initial findings and observations.

## Load Data

### Subtask:
Load the semiconductor calibration results from '/content/manufacturing_data.CSV' into a pandas DataFrame.


**Reasoning**:
First, I'll import the pandas library to enable data manipulation. Then, I'll load the specified CSV file into a DataFrame named `df` and display its first few rows to verify successful loading.



**Reasoning**:
The `ParserError` indicates malformed lines in the CSV file that are causing the loading to fail. By setting `on_bad_lines='skip'`, pandas will ignore these problematic lines, allowing the rest of the data to be loaded. Using `engine='python'` is necessary for `on_bad_lines` parameter to work.



In [ ]:
import pandas as pd
df = pd.read_csv('/content/HCT#100 FT CUC 1mohm 1.5% Antes.CSV', sep=None, engine='python', on_bad_lines='skip')
df.head()

## Initial Data Inspection

### Subtask:
Display the first few rows, column names, and data types to understand the structure of the calibration data.


**Reasoning**:
To understand the structure of the loaded data, I will display the first few rows, the column names, and their respective data types using pandas functions.



In [ ]:
print("First few rows of the DataFrame:")
print(df.head())
print("\nColumn names:")
print(df.columns)
print("\nData types and non-null values:")
df.info()

## Data Preprocessing (Correcting Data Loading)

### Subtask:
Reload the data, skipping initial metadata rows, to correctly load the tabular calibration results.


**Reasoning**:
To correctly load the tabular data, I need to skip the initial metadata rows. Based on the previous error, I will use `skiprows=15` to start reading the CSV from line 16, and then display the head, columns, and info of the new DataFrame to verify the loading.



In [ ]:
import pandas as pd
df_calibration = pd.read_csv('//content/HCT#100 FT CUC 1mohm 1.5% Antes.CSV', skiprows=15)
print("First few rows of the corrected DataFrame:")
print(df_calibration.head())
print("\nColumn names:")
print(df_calibration.columns)
print("\nData types and non-null values:")
df_calibration.info()

In [ ]:
import pandas as pd

# Step 1: Read the header row (line 16 of the original CSV) to extract column names
# skiprows=15 gets to line 16, nrows=1 reads only that line.
header_df = pd.read_csv('/content/HCT#100 FT CUC 1mohm 1.5% Antes.CSV', skiprows=15, nrows=1, header=None, engine='python')

# Extract potential column names from header_df, starting from the third column (index 2)
# The first column will be a test identifier, the second is often empty.
new_columns_raw = list(header_df.iloc[0, 2:].values)

# Ensure column names are unique by appending numerical suffixes
unique_columns = []
counts = {}
for col in new_columns_raw:
    col = str(col).strip()
    if col in counts:
        counts[col] += 1
        unique_columns.append(f"{col}_{counts[col]}")
    else:
        counts[col] = 0 # Initialize count for first occurrence
        unique_columns.append(col) # No suffix for the first occurrence

# The final list of column names: 'Test_Type' for the first column,
# 'Unnamed_Col' for the second (often empty), followed by the unique test names.
new_columns = ['Test_Type', 'Unnamed_Col'] + unique_columns

# Step 2: Load the actual calibration data
# Skip 30 rows to start reading from line 31 of the original CSV (where the actual numerical data begins).
# header=None indicates that there's no header row in the data being read at skiprows=30,
# and names=new_columns assigns our constructed column names.
df_calibration_cleaned = pd.read_csv(
    '/content/HCT#100 FT CUC 1mohm 1.5% Antes.CSV',
    skiprows=30, # Skip 15 (initial metadata) + 1 (header row) + 14 (further metadata) = 30 rows
    header=None, # No header row in the data part
    names=new_columns # Assign the unique column names
)

print("First few rows of the truly tabular DataFrame:")
print(df_calibration_cleaned.head())
print("\nColumn names:")
print(df_calibration_cleaned.columns)
print("\nData types and non-null values:")
df_calibration_cleaned.info()

## Calculate Descriptive Statistics

### Subtask:
Compute and display descriptive statistics (mean, standard deviation, min, max, etc.) for the numerical columns in the calibration data to get an overview of the value distributions.


**Reasoning**:
To get an overview of the value distributions, I will identify all numerical columns in the `df_calibration_cleaned` DataFrame and then calculate their descriptive statistics.



In [ ]:
print("Descriptive statistics for numerical columns:")
# Identify numerical columns
numerical_columns = df_calibration_cleaned.select_dtypes(include=['number'])

# Calculate descriptive statistics
descriptive_stats = numerical_columns.describe()

# Display the results
print(descriptive_stats)

## Visualize Value Distributions

### Subtask:
Generate histograms or box plots for key numerical columns to visualize the distribution of calibration values and identify any potential outliers or patterns. Include appropriate legends.


**Reasoning**:
To visualize the distribution of calibration values, I will generate histograms and box plots for the specified key numerical columns, ensuring each plot has appropriate labels and titles for clarity.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Select a few representative numerical columns for visualization
# Corrected selected_columns to use existing column names from '1' to '48'.
selected_columns = ['1', '2', '40', '45', '48']

# --- Generate Histograms ---
print("Generating Histograms for selected numerical columns...")
plt.figure(figsize=(15, 10))
for i, col in enumerate(selected_columns):
    plt.subplot(2, 3, i + 1) # Arrange in 2 rows, 3 columns
    sns.histplot(df_calibration_cleaned[col], kde=True)
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

# --- Generate Box Plots ---
print("\nGenerating Box Plots for selected numerical columns...")
plt.figure(figsize=(15, 10))
for i, col in enumerate(selected_columns):
    plt.subplot(2, 3, i + 1) # Arrange in 2 rows, 3 columns
    sns.boxplot(y=df_calibration_cleaned[col])
    plt.title(f'Box Plot of {col}')
    plt.ylabel(col)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

def load_calibration_data(file_path):
    """Load and process calibration data from a CSV file."""
    try:
        # Skip the first 15 rows to get to the relevant data
        data = pd.read_csv(file_path, skiprows=15, header=None)
        return data
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

def analyze_calibration(data, tolerance=0.015):
    """Analyze calibration data and check against tolerance."""
    results = []
    critical_tests = []

    for i, row in data.iterrows():
        if pd.notna(row[2]):  # Assuming the measurements start from the third column
            value = row[2]
            # Calculate error percentage
            # Here, you would typically compare against a reference value
            # For demonstration, assume reference is the first value
            if i == 0:
                reference = value
            error = abs(value - reference) / reference * 100

            if error > tolerance:
                critical_tests.append((i, error))

            results.append({
                'test_id': i,
                'value': value,
                'error': error,
                'within_tolerance': error <= tolerance
            })

    # Sort critical tests by descending error
    critical_tests.sort(key=lambda x: x[1], reverse=True)
    return results, critical_tests[:10]

def compare_calibrations(initial_data, adjusted_data):
    """Compare initial and adjusted calibration data."""
    initial_results, initial_critical = analyze_calibration(initial_data)
    adjusted_results, adjusted_critical = analyze_calibration(adjusted_data)

    # Calculate overall improvement metrics
    improvement = sum(1 for i, a in zip(initial_results, adjusted_results) if a['error'] < i['error'])
    avg_initial_error = sum(r['error'] for r in initial_results) / len(initial_results)
    avg_adjusted_error = sum(r['error'] for r in adjusted_results) / len(adjusted_results)

    return improvement, avg_initial_error, avg_adjusted_error, initial_critical, adjusted_critical

def visualize_critical_tests(initial_critical, adjusted_critical):
    """Visualize the most critical tests."""
    plt.figure(figsize=(10, 6))
    plt.title('Critical Tests Comparison')
    plt.subplot(1, 2, 1)
    plt.bar(range(len(initial_critical)), [err for _, err in initial_critical])
    plt.title('Initial Calibration')
    plt.xlabel('Test ID')
    plt.ylabel('Error (%)')

    plt.subplot(1, 2, 2)
    plt.bar(range(len(adjusted_critical)), [err for _, err in adjusted_critical])
    plt.title('Adjusted Calibration')
    plt.xlabel('Test ID')
    plt.ylabel('Error (%)')
    plt.tight_layout()
    plt.show()

def generate_report(initial_results, adjusted_results, critical_initial, critical_adjusted, engineer_name):
    """Generate a detailed report."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"calibration_analysis_{timestamp}.txt"

    with open(filename, 'w') as f:
        f.write(f"Calibration Analysis Report - {timestamp}\n")
        f.write(f"Engineer: {engineer_name}\n\n")

        f.write("Initial Calibration Results:\n")
        for result in initial_results:
            status = "Within tolerance" if result['within_tolerance'] else "OUT OF TOLERANCE"
            f.write(f"Test {result['test_id']}: Value={result['value']}, Error={result['error']:.2f}% {status}\n")

        f.write("\nAdjusted Calibration Results:\n")
        for result in adjusted_results:
            status = "Within tolerance" if result['within_tolerance'] else "OUT OF TOLERANCE"
            f.write(f"Test {result['test_id']}: Value={result['value']}, Error={result['error']:.2f}% {status}\n")

        f.write("\nCritical Tests Initial:\n")
        for test_id, error in critical_initial:
            f.write(f"Test {test_id}: {error:.2f}%\n")

        f.write("\nCritical Tests Adjusted:\n")
        for test_id, error in critical_adjusted:
            f.write(f"Test {test_id}: {error:.2f}%\n")

        f.write("\nSummary:\n")
        f.write(f"Total Tests: {len(initial_results)}\n")
        f.write(f"Initial Out of Tolerance: {len([r for r in initial_results if not r['within_tolerance']])}\n")
        f.write(f"Adjusted Out of Tolerance: {len([r for r in adjusted_results if not r['within_tolerance']])}\n")

    print(f"Report saved as {filename}")

def main():
    print("Calibration Analysis Tool")
    print("-------------------------")

    initial_data = None
    adjusted_data = None

    while True:
        print("\nMenu:")
        print("1. Load Initial Calibration Data")
        print("2. Load Adjusted Calibration Data")
        print("3. Analyze Initial Calibration")
        print("4. Analyze Adjusted Calibration")
        print("5. Compare Calibrations")
        print("6. Visualize Critical Tests")
        print("7. Generate Report")
        print("8. Exit")

        choice = input("Choose an option: ")

        if choice == '1':
            file_path = input("Enter path to initial calibration file: ")
            initial_data = load_calibration_data(file_path)
            if initial_data is not None:
                print("Initial calibration data loaded successfully.")
        elif choice == '2':
            file_path = input("Enter path to adjusted calibration file: ")
            adjusted_data = load_calibration_data(file_path)
            if adjusted_data is not None:
                print("Adjusted calibration data loaded successfully.")
        elif choice == '3':
            if initial_data is not None:
                results, critical = analyze_calibration(initial_data)
                print("\nInitial Calibration Analysis:")
                for result in results:
                    print(f"Test {result['test_id']}: {result['value']}, Error={result['error']:.2f}% { 'OK' if result['within_tolerance'] else 'FAIL' }")
            else:
                print("No initial calibration data loaded.")
        elif choice == '4':
            if adjusted_data is not None:
                results, critical = analyze_calibration(adjusted_data)
                print("\nAdjusted Calibration Analysis:")
                for result in results:
                    print(f"Test {result['test_id']}: {result['value']}, Error={result['error']:.2f}% { 'OK' if result['within_tolerance'] else 'FAIL' }")
            else:
                print("No adjusted calibration data loaded.")
        elif choice == '5':
            if initial_data is not None and adjusted_data is not None:
                improvement, initial_avg, adjusted_avg, initial_critical, adjusted_critical = compare_calibrations(initial_data, adjusted_data)
                print(f"\nImprovement: {improvement} tests improved")
                print(f"Initial Average Error: {initial_avg:.2f}%")
                print(f"Adjusted Average Error: {adjusted_avg:.2f}%")
            else:
                print("Both datasets must be loaded to compare.")
        elif choice == '6':
            if initial_data is not None and adjusted_data is not None:
                visualize_critical_tests(initial_data, adjusted_data)
            else:
                print("Both datasets must be loaded to visualize.")
        elif choice == '7':
            engineer_name = input("Enter your name: ")
            if initial_data is not None and adjusted_data is not None:
                initial_results, initial_critical = analyze_calibration(initial_data)
                adjusted_results, adjusted_critical = analyze_calibration(adjusted_data)
                generate_report(initial_results, adjusted_results, initial_critical, adjusted_critical, engineer_name)
            else:
                print("Both datasets must be loaded to generate report.")
        elif choice == '8':
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please choose a valid option.")

if __name__ == "__main__":
    main()

# Task
Refine the data loading and analysis process by updating the interactive menu to prompt the user for the data type (Alta Corriente, Mainframe, or Produccion) when loading initial or adjusted calibration files, and ensure that the `load_calibration_data` function correctly parses the selected file based on its specified type.

## Update Interactive Menu for File Type Selection

### Subtask:
Integrate a new prompt into the 'Load Initial Calibration Data' and 'Load Adjusted Calibration Data' options within the `main` function (interactive menu). Before file upload or path entry, the user will be asked to select the type of data being loaded: 'Alta Corriente', 'Mainframe', or 'Produccion'.


**Reasoning**:
To implement the subtask, I need to modify the `main` function to prompt the user for the data type and pass it to the `load_calibration_data` function. I also need to update the `load_calibration_data` function to accept this new parameter and, more importantly, to correctly parse the numerical data from the CSV, converting string values to numeric to fix the `TypeError` observed previously. Finally, I will adjust the `visualize_critical_tests` function to correctly use the output from `analyze_calibration`.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configuración inicial
TOLERANCE_PERCENT = 1.5  # Tolerancia por defecto

# Función para cargar y procesar los datos
def load_calibration_data(file_path, file_type='Alta Corriente'):
    """Cargar y procesar datos de calibración de diferentes formatos."""
    try:
        if file_type == 'Mainframe':
            header_df = pd.read_csv(file_path, skiprows=25, nrows=1, header=None, sep=';', engine='python')
            data = pd.read_csv(file_path, skiprows=30, header=None, sep=';', engine='python')
        else:
            header_df = pd.read_csv(file_path, skiprows=15, nrows=1, header=None, engine='python')
            data = pd.read_csv(file_path, skiprows=30, header=None, engine='python')

        # Procesar nombres de columnas dinámicamente
        new_columns_raw = list(header_df.iloc[0, 2:].values)
        unique_columns = []
        counts = {}
        for col in new_columns_raw:
            col = str(col).strip()
            if col in counts:
                counts[col] += 1
                unique_columns.append(f"{col}_{counts[col]}")
            else:
                counts[col] = 0
                unique_columns.append(col)
        new_columns = ['Test_Type', 'Unnamed_Col'] + unique_columns

        # Renombrar columnas
        data.columns = new_columns

        # Convertir columnas relevantes a numéricas
        for col in new_columns[2:]:
            data[col] = pd.to_numeric(data[col], errors='coerce')

        return data
    except Exception as e:
        print(f"Error al cargar el archivo {file_type}: {e}")
        return None

# Función para analizar la calibración con tolerancia personalizada
def analyze_calibration(data, tolerance=TOLERANCE_PERCENT):
    """Analizar calibración con una tolerancia específica."""
    results = []
    critical_tests = []
    stats = {'total_tests': 0, 'within_tolerance': 0, 'out_of_tolerance': 0}

    if data is None or not isinstance(data, pd.DataFrame):
        print("Datos de calibración inválidos.")
        return results, critical_tests, stats

    reference = None
    for index, row in data.iterrows():
        if pd.notna(row.iloc[2]):
            value = row.iloc[2]
            stats['total_tests'] += 1
            if reference is None:
                reference = value

            if reference == 0:
                error = float('inf') if value != 0 else 0
            else:
                error = abs(value - reference) / reference * 100

            status = "Dentro de tolerancia" if error <= tolerance else "Fuera de tolerancia"
            results.append({
                'test_id': index,
                'value': value,
                'error': error,
                'status': status
            })

            if error > tolerance:
                critical_tests.append((index, error))
                stats['out_of_tolerance'] += 1
            else:
                stats['within_tolerance'] += 1

    stats['avg_error'] = sum(r['error'] for r in results) / len(results) if results else 0
    critical_tests.sort(key=lambda x: x[1], reverse=True)
    return results, critical_tests[:10], stats

# Función para comparar calibraciones
def compare_calibrations(initial_data, adjusted_data):
    """Comparar calibraciones iniciales y ajustadas."""
    initial_results, initial_critical, initial_stats = analyze_calibration(initial_data)
    adjusted_results, adjusted_critical, adjusted_stats = analyze_calibration(adjusted_data)

    comparison_stats = {
        'improvement': sum(1 for i, a in zip(initial_results, adjusted_results) if a['error'] < i['error']),
        'initial_avg_error': initial_stats['avg_error'],
        'adjusted_avg_error': adjusted_stats['avg_error'],
        'initial_out_of_tolerance': initial_stats['out_of_tolerance'],
        'adjusted_out_of_tolerance': adjusted_stats['out_of_tolerance'],
        'differences': [],
        'similarities': []
    }

    for i, a in zip(initial_results, adjusted_results):
        if i['test_id'] == a['test_id']:
            if i['status'] != a['status']:
                comparison_stats['differences'].append(f"Test {i['test_id']}: {i['status']} -> {a['status']}")
            else:
                comparison_stats['similarities'].append(f"Test {i['test_id']}: {i['status']} (Sin cambios)")

    return comparison_stats

# Función para visualizar comparación
def visualize_comparison(comparison_stats):
    """Visualizar comparación de calibraciones."""
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.bar(['Inicio', 'Fin'], [comparison_stats['initial_avg_error'], comparison_stats['adjusted_avg_error']])
    plt.title('Error Promedio')
    plt.xlabel('Etapa')
    plt.ylabel('Error (%)')

    plt.subplot(1, 2, 2)
    plt.bar(['Errores iniciales', 'Errores finales'], [comparison_stats['initial_out_of_tolerance'], comparison_stats['adjusted_out_of_tolerance']])
    plt.title('Número de Errores Fuera de Tolerancia')
    plt.xlabel('Etapa')
    plt.ylabel('Cantidad')

    plt.tight_layout()
    plt.show()

# Función para generación de informe
def generate_report(comparison_stats, engineer_name):
    """Generar informe detallado de comparación."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"calibration_comparison_report_{timestamp}.txt"

    with open(filename, 'w') as f:
        f.write(f"Informe de Comparación de Calibración - {timestamp}\n")
        f.write(f"Ingeniero: {engineer_name}\n\n")

        f.write("Estadísticas de Calibración Inicial:\n")
        for key, value in comparison_stats['initial_stats'].items():
            f.write(f"{key.title().replace('_', ' ')}: {value}\n")

        f.write("\nEstadísticas de Calibración Ajustada:\n")
        for key, value in comparison_stats['adjusted_stats'].items():
            f.write(f"{key.title().replace('_', ' ')}: {value}\n")

        f.write("\nDiferencias:\n")
        for difference in comparison_stats['differences']:
            f.write(f"- {difference}\n")

        f.write("\nSimilitudes:\n")
        for similarity in comparison_stats['similarities']:
            f.write(f"- {similarity}\n")

        f.write(f"\nMejora en la Calibración:\n")
        f.write(f"Número de pruebas mejoradas: {comparison_stats['improvement']}\n")
        f.write(f"Redución promedio del error: {comparison_stats['initial_avg_error'] - comparison_stats['adjusted_avg_error']:.2f}%\n")

    print(f"Informe generado correctamente: {filename}")

# Función para visualización de SPC
def visualize_spccomparison(initial_critical, adjusted_critical):
    """Visualizar comparación de SPC."""
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    if initial_critical:
        test_ids_initial = [str(t_id) for t_id, _ in initial_critical]
        errors_initial = [err for _, err in initial_critical]
        plt.bar(test_ids_initial, errors_initial, color='skyblue')
        plt.title('Pruebas Iniciales Críticas')
        plt.xlabel('ID de Prueba')
        plt.ylabel('Error (%)')
    else:
        plt.text(0.5, 0.5, "No hay pruebas iniciales críticas.", ha='center', va='center', transform=plt.gca().transAxes)

    plt.subplot(1, 2, 2)
    if adjusted_critical:
        test_ids_adjusted = [str(t_id) for t_id, _ in adjusted_critical]
        errors_adjusted = [err for _, err in adjusted_critical]
        plt.bar(test_ids_adjusted, errors_adjusted, color='lightcoral')
        plt.title('Pruebas Ajustadas Críticas')
        plt.xlabel('ID de Prueba')
        plt.ylabel('Error (%)')
    else:
        plt.text(0.5, 0.5, "No hay pruebas ajustadas críticas.", ha='center', va='center', transform=plt.gca().transAxes)

    plt.tight_layout()
    plt.show()

# Función para correlación
def calculate_correlation(data):
    """Calcular correlación entre columnas numéricas."""
    numerical_data = data.select_dtypes(include=['int64', 'float64'])
    correlation_matrix = numerical_data.corr()
    return correlation_matrix

def visualize_correlation(correlation_matrix):
    """Visualizar matriz de correlación."""
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', square=True)
    plt.title('Matriz de Correlación')
    plt.show()

# Manejo de errores mejorado
def handle_errors(func):
    """Decorador para manejar errores."""
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"Error en {func.__name__}: {e}")
            return None
    return wrapper

# Menú interactivo
def main():
    print("Herramienta de Análisis de Calibración")
    print("------------------------------------")

    initial_data = None
    adjusted_data = None

    while True:
        print("\nMenú:")
        print("1. Cargar Datos de Calibración Inicial")
        print("2. Cargar Datos de Calibración Ajustada")
        print("3. Analizar Calibración Inicial")
        print("4. Analizar Calibración Ajustada")
        print("5. Comparar Calibraciones")
        print("6. Visualizar SPC")
        print("7. Generar Informe")
        print("8. Visualizar Correlación")
        print("9. Salir")

        choice = input("Seleccione una opción: ")

        if choice == '1':
            file_type = input("Seleccione el tipo de archivo ('Alta Corriente', 'Mainframe'): ")
            file_path = input("Ingrese la ruta del archivo de calibración inicial: ")
            initial_data = load_calibration_data(file_path, file_type)
            if initial_data is not None:
                print("Datos cargados correctamente.")
        elif choice == '2':
            file_type = input("Seleccione el tipo de archivo ('Alta Corriente', 'Mainframe'): ")
            file_path = input("Ingrese la ruta del archivo de calibración ajustada: ")
            adjusted_data = load_calibration_data(file_path, file_type)
            if adjusted_data is not None:
                print("Datos cargados correctamente.")
        elif choice == '3':
            if initial_data is not None:
                results, critical, stats = analyze_calibration(initial_data)
                print("\nResultados de la calibración inicial:")
                for result in results:
                    print(f"Prueba {result['test_id']}: {result['value']}, Error={result['error']:.2f}% {result['status']}")
                print("\nEstadísticas:")
                for key, value in stats.items():
                    print(f"{key.title().replace('_', ' ')}: {value}")
            else:
                print("No se han cargado los datos de calibración inicial.")
        elif choice == '4':
            if adjusted_data is not None:
                results, critical, stats = analyze_calibration(adjusted_data)
                print("\nResultados de la calibración ajustada:")
                for result in results:
                    print(f"Prueba {result['test_id']}: {result['value']}, Error={result['error']:.2f}% {result['status']}")
                print("\nEstadísticas:")
                for key, value in stats.items():
                    print(f"{key.title().replace('_', ' ')}: {value}")
            else:
                print("No se han cargado los datos de calibración ajustada.")
        elif choice == '5':
            if initial_data is not None and adjusted_data is not None:
                comparison_stats = compare_calibrations(initial_data, adjusted_data)
                print("\nEstadísticas de comparación:")
                print(f"Mejora en pruebas: {comparison_stats['improvement']}")
                print(f"Error promedio inicial: {comparison_stats['initial_avg_error']:.2f}%")
                print(f"Error promedio ajustado: {comparison_stats['adjusted_avg_error']:.2f}%")
                visualize_comparison(comparison_stats)
            else:
                print("Necesita cargar ambos conjuntos de datos para realizar la comparación.")
        elif choice == '6':
            if initial_data is not None and adjusted_data is not None:
                initial_stats = analyze_calibration(initial_data)[2]
                initial_critical = analyze_calibration(initial_data)[1]
                adjusted_stats = analyze_calibration(adjusted_data)[2]
                adjusted_critical = analyze_calibration(adjusted_data)[1]
                visualize_spccomparison(initial_critical, adjusted_critical)
            else:
                print("Necesita cargar ambos conjuntos de datos para visualizar el SPC.")
        elif choice == '7':
            engineer_name = input("Ingrese su nombre: ")
            if initial_data is not None and adjusted_data is not None:
                comparison_stats = compare_calibrations(initial_data, adjusted_data)
                generate_report(comparison_stats, engineer_name)
            else:
                print("Necesita cargar ambos conjuntos de datos para generar el informe.")
        elif choice == '8':
            if initial_data is not None:
                correlation_matrix = calculate_correlation(initial_data)
                visualize_correlation(correlation_matrix)
            else:
                print("Necesita cargar los datos para calcular la correlación.")
        elif choice == '9':
            print("Saliendo del programa.")
            break
        else:
            print("Opción inválida. Por favor seleccione una opción válida.")

if __name__ == "__main__":
    main()